In [1]:
import pandas as pd

In [2]:
sf = pd.read_csv('ca_san_francisco_2020_04_01.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sf.head(15)

,raw_row_number,date,time,location,lat,lng,district,subject_age,subject_race,subject_sex,...,citation_issued,warning_issued,outcome,contraband_found,search_conducted,search_vehicle,search_basis,reason_for_stop,raw_search_vehicle_description,raw_result_of_contact_description
0,869921,2014-08-01,00:01:00,MASONIC AV & FELL ST,37.773004,-122.445873,NaN,NaN,asian/pacific islander,female,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning
1,869922,2014-08-01,00:01:00,GEARY&10TH AV,37.780898,-122.468586,NaN,NaN,black,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
2,869923,2014-08-01,00:15:00,SUTTER N OCTAVIA ST,37.786919,-122.426718,NaN,NaN,hispanic,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
3,869924,2014-08-01,00:18:00,3RD ST & DAVIDSON,37.746380,-122.392005,NaN,NaN,hispanic,male,...,False,True,warning,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Warning
4,869925,2014-08-01,00:19:00,DIVISADERO ST. & BUSH ST.,37.786348,-122.440003,NaN,NaN,white,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
5,869926,2014-08-01,00:30:00,3RD ST AND GALVEZ AVE,37.741082,-122.388433,NaN,NaN,black,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
6,869927,2014-08-01,00:30:00,COLUMBUS AND BROADWAY,37.797875,-122.406716,NaN,NaN,hispanic,male,...,True,False,citation,NaN,False,False,NaN,Moving Violation,No Search,Citation
7,869928,2014-08-01,00:35:00,INGALLS & QUESADA,37.729122,-122.384013,NaN,NaN,black,female,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
8,869929,2014-08-01,01:00:00,17TH ST & BRYANT ST,37.764318,-122.410384,NaN,NaN,black,male,...,True,False,citation,NaN,False,False,NaN,Mechanical or Non-Moving Violation (V.C.),No Search,Citation
9,869930,2014-08-01,01:00:00,FULTON ST. & FUNSTON AV.,37.773166,-122.471264,NaN,NaN,white,female,...,True,False,citation,NaN,False,False,NaN,Moving Violation,No Search,Citation


In [4]:
sf.outcome.value_counts(normalize=True)

citation    0.710153
warning     0.276439
arrest      0.013408
Name: outcome, dtype: float64

In [5]:
arrest = sf[sf.outcome == 'arrest']

In [6]:
arrest.subject_race.value_counts(normalize=True)

white                     0.332914
black                     0.296855
hispanic                  0.191530
asian/pacific islander    0.102390
other                     0.076310
Name: subject_race, dtype: float64

In [7]:
arrest.subject_age.describe()

count    11320.000000
mean        34.579682
std         12.357852
min         10.000000
25%         25.000000
50%         32.000000
75%         42.000000
max         99.000000
Name: subject_age, dtype: float64

In [9]:
youth_arrests = arrest[arrest.subject_age < 25]

In [10]:
youth.subject_race.value_counts(normalize=True)

black                     0.338609
white                     0.254693
hispanic                  0.224512
asian/pacific islander    0.104159
other                     0.078027
Name: subject_race, dtype: float64